### Imports

In [1]:
from __future__ import annotations

from pathlib import Path

from kub.course.plotlib.simulationPlotFactory import SimulationPlotFactory
from kub.course.simlib.simulation import FMUSimulation

# Find repository root: check if database/ exists at current location
repo_root = (
    Path.cwd() if (Path.cwd() / "database").exists() else Path.cwd().parent.parent
)
databasePath = repo_root / "database" / "day1" / "FMUs"

### **Exercise 7: Infrared Radiation**

<img src="../../database/images/InfraredRadiation7.png"
     align="right"
     width="400"
     style="margin-right: 20px; margin-bottom: 10px;" />

This exercise aims to **quantify the radiative exchange** between building envelope surfaces (walls and roofs) and the outdoor environment (ground and sky).

The **net radiative power** exchanged by a surface is calculated using the **Stefan-Boltzmann Law** and **View Factors (F)**

$\phi_{QflowWall} = \epsilon . \sigma . A(F_{wall,Sky} . T_{Sky}^4 + F_{wall,Ground} . T_{Ground}^4 - T_{wall}^4)$

$\phi_{QflowRoof} = \epsilon . \sigma . A(F_{Roof,Sky} . T_{Sky}^4 + F_{Roof,Ground} . T_{Ground}^4 - T_{wall}^4)$


**<ins>Question 1:<ins>**

Consider a rectangular building on flat ground. **Determine the view factor (F)** for a **vertical wall** and a **flat roof** relative to the environment (sky and ground).

**<ins>Question 2:<ins>**

Using the provided simulation, **observe the phenomenon where the wall temperature drops below the ambient air temperature**. Explain empirically why this occurrence is **thermodynamically impossible** without accounting for **infrared radiation**.

In [6]:
sim1 = FMUSimulation(databasePath / "Exercises_Infra_Red_Radiation_7.fmu")

sim1.initialize(startTime=0.0, stopTime=86400 * 4, timeStep=3600.0)
sim1.initParameters(
    {"viewFactorWall": 0.2, "viewFactorRoof": 0.2}
)  # modify view factors
sim1.exitInitialization()

outputNames = [
    "WallLw",
    "RoofLw",
    "ambience.TAirRef",
    "ambience.TSky",
    "building.wall_WEST.toSurfacePort_1.heatPort.T",
]
data1 = sim1.run(outputNames)
sim1.terminate()

sim1Factory = SimulationPlotFactory()

sim1Factory.plot_multi_curves(
    time=data1["time"],
    data_type="heat_flux",
    data_dict={"WallLw": data1["WallLw"], "RoofLw": data1["RoofLw"]},
    title="Exercise 7 — Long Wave Radiation received by Wall and Roof",
)

sim1Factory.plot_multi_curves(
    time=data1["time"],
    data_type="temperature",
    data_dict={
        "Sky temperature": data1["ambience.TSky"],
        "Air temperature on reference height": data1["ambience.TAirRef"],
        "Wall west interior temperature": data1[
            "building.wall_WEST.toSurfacePort_1.heatPort.T"
        ],
    },
    title="Exercise 7 — Temperature evolution during Simulation",
);